# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder              #使用標籤編碼對於標籤做預處理
from sklearn.model_selection import cross_val_score         #對於模型選擇，利用交叉驗證評分
from sklearn.linear_model import LogisticRegression         #估測器模型選用線性回歸

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [3]:
# 觀察欄位相異值數量
#select_dtypes(include=["object"])==>選出型態為object的欄位
#pd.unique(Series)获取Series中元素的唯一值（即去掉重复的）
#nunique只作用于Series,用法是Series.nunique()，返回Series中只元素的唯一值的統計次數
df.select_dtypes(include=["object"]).apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

# 作業2
* 承上題，三者比較效果何者最好?

In [4]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()    #設一個空的dataframe，名稱叫做df_temp
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])   #每一個column依序做一次標籤編碼，c欄位資料用fit_transform完成後，塞入df_temp之c欄位
train_X = df_temp[:train_num]            #只取0~(train_num-1)之訓練集資料，不包含第train_num個以後的測試及資料           
estimator = LogisticRegression()         #估測器選用線性迴歸
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())   #cv=5-->計算5次分數，再取mean()得到平均分數
df_temp.head()                           #每一個column，是原本數值做標籤編碼後的編碼結果

0.780004837244799


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [5]:
"""
df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'})
會以df之Cabin為分類基準運算完後，Cabin會被放在index項
其中，運算是以name這個column做運算，會計算每個Cabin項目有幾個name，也就是計算每個cabin有幾個人
計算結果這個column原本是叫name，現在改叫做Cabin_count
    -->Cabin_count會計算每個cabin有幾個人

接著用.reset_index()把index改成0、1、2、3....，再把ticket移到第0個column
"""
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
#count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'count'}).reset_index()
#count_df 
#count_df 會有兩個column，一個是cabin名稱，一個是每一個cabin有幾個人住

# 將df與count_df兩個dataframe merge合併, 合併是基於 'cabin' 欄位
# 使用 how='left' 是左邊的dataframe，也就是df的所有 index 與順序不改變
df = pd.merge(df, count_df, on=['Cabin'], how='left')
df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014
5,"Moran, Mr. James",male,330877,None,Q,1014
6,"McCarthy, Mr. Timothy J",male,17463,E46,S,2
7,"Palsson, Master. Gosta Leonard",male,349909,None,S,1014
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,None,S,1014
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,None,C,1014


In [6]:
#count_df有兩個dataframe，一個是cabin名稱，一個是各個cabin裡面的住戶人數
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)  #依照'Ticket_Count'這一列排序
                                                                     #ascending=False表示依據降序排列
                                                                     #inplace默認是False，用排序后的数据不會替换原来的数据

,Cabin,Cabin_Count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [7]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()    #建立一個空的dataframe，名稱叫做df_temp
for c in object_features:   #將型態為object的這幾個column做標籤編碼，每個column依序輸入再利用fit_transform完成標籤編碼後，輸出到df_temp的標籤c之下
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Count'] = df['Cabin_Count'] #df的'Cabin_Count'之下有計算好的'Cabin'之計數編碼，輸出到df_temp的標籤'Cabin_Count'之下
train_X = df_temp[:train_num]              #training data只取第0到(train_num-1)之訓練集部分，其他的測試集部分不要取
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())  #training data是train_X，正確的目標標籤是train_Y，評估5次分數後，再算平均分數
df_temp.head()

0.7856230275549181


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


In [12]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()             #設定一個空的dataframe
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
# 這邊的雜湊編碼, 是直接將 'Ticket' 的名稱放入雜湊函數的輸出數值, 為了要確定是緊密(dense)特徵, 因此除以10後看餘數
# 這邊的 10 是隨機選擇, 不一定要用 10, 同學可以自由選擇購小的數字試看看. 基本上效果都不會太好
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 100)  #運算時以df之Cabin這個column之series數值輸入到函數中
                                                                #.map表示函數的計算是以series之每個元素當輸入，也就是每個元素自行運算
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7788875188090448


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,83
1,286,0,816,106,0,68
2,523,0,914,185,3,83
3,422,0,65,70,3,40
4,22,1,649,185,3,83


In [13]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
# 這邊的雜湊編碼, 是直接將 'Ticket' 的名稱放入雜湊函數的輸出數值, 為了要確定是緊密(dense)特徵, 因此除以10後看餘數
# 這邊的 10 是隨機選擇, 不一定要用 10, 同學可以自由選擇購小的數字試看看. 基本上效果都不會太好
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 100)
df_temp['Cabin_Count'] = df['Cabin_Count']

train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7878766374907041


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash,Cabin_Count
0,155,1,720,185,3,83,1014
1,286,0,816,106,0,68,2
2,523,0,914,185,3,83,1014
3,422,0,65,70,3,40,2
4,22,1,649,185,3,83,1014


--------------------
Ans.'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸效果最好